In [8]:
#Basics:
#https://colab.research.google.com/drive/1cqaMNrVILqPtvkokwTyt-ZvHP0Er7LM2?usp=sharing
#network
#https://colab.research.google.com/drive/1WXVMGQ1N0k8SJlXw1tkYkKAsyjHOhGO_?usp=sharing
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import urllib

In [9]:
#same with resnet152, googlenet
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/American_Eskimo_Dog.jpg/1024px-American_Eskimo_Dog.jpg"
filename = "dog.jpg"
urllib.request.urlretrieve(url, filename)
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [10]:
model.eval()
with torch.no_grad():
    output = model(input_batch)

print(output.shape)
probabilities = F.softmax(output[0], dim=0)
print(probabilities.sum())

torch.Size([1, 1000])
tensor(1.0000)


In [11]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

categories = []
with open("imagenet_classes.txt", "r") as f:
    for s in f.readlines():
        categories.append(s.strip())

--2024-05-02 16:32:12--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: 'imagenet_classes.txt.1'

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-05-02 16:32:12 (57.2 MB/s) - 'imagenet_classes.txt.1' saved [10472/10472]



In [12]:
top5_p, top5_c = torch.topk(probabilities, 5)
for i in range(top5_p.size(0)):
    print(categories[top5_c[i]], top5_p[i].item())

Samoyed 0.8408327102661133
Pomeranian 0.03436620905995369
white wolf 0.017395561560988426
Eskimo dog 0.016164731234312057
Great Pyrenees 0.010918729938566685
